In [ ]:
# Align dates to Month End to ensure matching with df_global
def align_to_month_end(df):
    df_aligned = df.copy()
    # Convert to period 'M' and back to timestamp 'M' (Month End)
    df_aligned.index = pd.to_datetime(df_aligned.index).to_period('M').to_timestamp('M')
    return df_aligned

# Align the source matrices
z_score_aligned = align_to_month_end(z_score_matrix)
lowvol_aligned = align_to_month_end(lowvol_matrix)

print("Updating df_global with Momentum and Defensive scores...")

# 1. Update Momentum
# Iterate through tickers that exist in both
common_tickers_mom = z_score_aligned.columns.intersection(df_global.columns.get_level_values('ticker').unique())
print(common_tickers_mom)
for ticker in common_tickers_mom:
    # Reindex the source series to match df_global's index
    # This aligns dates and handles any missing/extra dates
    series_aligned = z_score_aligned[ticker].reindex(df_global.index)
    df_global.loc[:, (ticker, "momentum")] = series_aligned

# 2. Update Defensive (Low Vol)
common_tickers_def = lowvol_aligned.columns.intersection(df_global.columns.get_level_values('ticker').unique())

for ticker in common_tickers_def:
    series_aligned = lowvol_aligned[ticker].reindex(df_global.index)
    df_global.loc[:, (ticker, "defensive")] = series_aligned

print("Update complete.")

# Verify
print("\nSample of df_global (Momentum & Defensive):")
# Pick a ticker that likely has data
if len(common_tickers_mom) > 0:
    sample_ticker = common_tickers_mom[30]
    print(f"Ticker: {sample_ticker}")
    print(df_global.loc[:, (sample_ticker, ["momentum", "defensive"])].dropna().head())
else:
    print("No common tickers found.")

In [ ]:
# Calculate the composite signal by averaging the 4 factors
# We take the mean across the 'factor' level (level 1) of the columns
# This will result in a DataFrame with Date index and Ticker columns
df_signal = df_global.groupby(level='ticker', axis=1).mean()

print("Signal DataFrame created.")
print(f"Shape: {df_signal.shape}")
print("\nSample of df_signal (First 5 rows, first 5 columns):")
print(df_signal.iloc[:5, :5])

# Optional: Check for missing values
print(f"\nTotal missing values: {df_signal.isna().sum().sum()}")
print(f"Percentage of missing values: {df_signal.isna().sum().sum() / df_signal.size * 100:.2f}%")